Importing all the Libraries to read the PDF texts and tables

In [ ]:
%pip install -r requirements.txt


import fitz  # PyMuPDF for text extraction
import pdfplumber  # For extracting tables
import pandas as pd
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


A Function to extract the texts from the pdf

In [2]:
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF using PyMuPDF."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

A Function to extract all the tables from the pdf

In [3]:
def extract_tables_from_pdf(pdf_path):
    """Extracts tables from a PDF using pdfplumber and returns them as DataFrames."""
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                df = pd.DataFrame(table)  # Convert table to DataFrame
                tables.append(df)
    return tables


In [4]:
def process_pdf(pdf_path):
    """Extracts both text and tables from a given PDF."""
    text = extract_text_from_pdf(pdf_path)
    tables = extract_tables_from_pdf(pdf_path)

    # Print extracted text
    print("\n=== Extracted Text ===\n")
    print(text[:1000])  # Show first 1000 characters
    print("\n... [Truncated] ...\n")

    # Print extracted tables
    if tables:
        print("\n=== Extracted Tables ===\n")
        for i, df in enumerate(tables):
            print(f"Table {i+1}:")
            print(df)
            print("\n")
    else:
        print("No tables found in the PDF.")

    return text, tables


In [5]:
pdf_path = "/Users/rakesh/Downloads/LLM_Project/PDF_Reader/greggs-2024-preliminary-results.pdf"
extracted_text, extracted_tables = process_pdf(pdf_path)

table_texts = "\n\n".join([df.to_csv(index=False, sep="|") for df in extracted_tables])




=== Extracted Text ===

1 
 
 
4 March 2025 
GREGGS PLC 
 
(“Greggs” or the “Company”) 
 
PRELIMINARY RESULTS FOR THE 52 WEEKS ENDED 28 DECEMBER 2024 
 
Strong progress, with milestone £2 billion sales and record profits 
 
2024 Financial highlights 
 
 
2024 
2023 
% 
Increase 
Total sales 
£2,014m £1,810m 
+11.3% 
Underlying operating profit excluding 
exceptional income* 
£195.3m £171.7m 
+13.7% 
Underlying pre-tax profit excluding exceptional 
income* 
£189.8m £167.7m 
+13.2% 
Pre-tax profit 
£203.9m £188.3m 
+8.3% 
Underlying diluted earnings per share* 
137.5p 
123.8p 
+11.1% 
Diluted earnings per share 
149.6p 
139.2p 
+7.5% 
Total ordinary dividend per share 
69.0p 
62.0p 
+11.3% 
 
* Excludes impact of £14.1 million exceptional net income primarily related to the sale of a legacy supply chain site (2023: £20.6 
million net income related to settlement of business interruption insurance claims made in 2020) 
 
• 
 Total sales** up 11.3% on 2023 level, with LFL*** sales in comp

In [6]:
load_dotenv(override=True)
my_api_key = os.getenv('OPENAI_API_KEY')

if not my_api_key:
    print("No API key was found ")
else:
    print("API key found! ")

openai = OpenAI()

API key found! 


In [7]:
system_prompt = "You are an expert financial assistant that analyses the contents of a stock Report\
and provides a short Report,\
Respond in markdown."

In [8]:
def user_prompt_for(extracted_text,table_texts):
    user_prompt = """You are an expert financial analyst, Based on the given stock report data, 
    generate a detailed stock analysis report that a newbie user can understand. Provide insights trends, and any key risks or opportunities.
    Inform the user whether to buy or sell the stock and why strictly based on the insights of the report. 
     \n\n
   """
    user_prompt += extracted_text
    user_prompt += table_texts
    return user_prompt

user_prompt = user_prompt_for(extracted_text,table_texts)



In [9]:
def messages_for(system_prompt, user_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [10]:
messages_for(system_prompt,user_prompt)

[{'role': 'system',
  'content': 'You are an expert financial assistant that analyses the contents of a stock Reportand provides a short Report,Respond in markdown.'},
 {'role': 'user',
  'content': 'You are an expert financial analyst, Based on the given stock report data, \n    generate a detailed stock analysis report that a newbie user can understand. Provide insights trends, and any key risks or opportunities.\n    Inform the user whether to buy or sell the stock and why strictly based on the insights of the report. \n     \n\n\n   1 \n \n \n4 March 2025 \nGREGGS PLC \n \n(“Greggs” or the “Company”) \n \nPRELIMINARY RESULTS FOR THE 52 WEEKS ENDED 28 DECEMBER 2024 \n \nStrong progress, with milestone £2 billion sales and record profits \n \n2024 Financial highlights \n \n \n2024 \n2023 \n% \nIncrease \nTotal sales \n£2,014m £1,810m \n+11.3% \nUnderlying operating profit excluding \nexceptional income* \n£195.3m £171.7m \n+13.7% \nUnderlying pre-tax profit excluding exceptional \nin

In [11]:
def stock_report(system_prompt,user_prompt):
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(system_prompt,user_prompt)
    )
    return response.choices[0].message.content

In [12]:
def display_summary(system_prompt,user_prompt):
    summary = stock_report(system_prompt,user_prompt)
    display(Markdown(summary))

In [13]:
display_summary(system_prompt,user_prompt)

# Greggs PLC Stock Analysis Report

**Date:** 4 March 2025

## Overview
Greggs PLC has delivered robust financial performance, marked by record sales of £2 billion for FY2024 and significant growth in profits. This analysis provides insights into the company's financial performance, strategic initiatives, and potential risks moving forward.

## Key Financial Highlights
- **Total Sales:** £2,014m, up **11.3%** from the previous year.
- **Underlying Operating Profit:** £195.3m, an increase of **13.7%**.
- **Underlying Pre-Tax Profit:** £189.8m, up **13.2%**.
- **Pre-Tax Profit:** £203.9m, increasing by **8.3%**.
- **Earnings per Share (EPS):** 
  - Underlying dilutive EPS: **137.5p** (+11.1%)
  - Dilutive EPS: **149.6p** (+7.5%)
- **Dividend per Share:** Total ordinary dividend increased to **69.0p**, up **11.3%** from last year.

## Strategic Progress and Insights
1. **Expansion of Shop Network:**
   - A record *226 new shops* were opened in 2024, bringing the total to 2,618.
   - Plans for an additional *140-150 net openings* in 2025 demonstrate a proactive approach towards market penetration.

2. **Growth in Sales Channels:**
   - Like-for-like (LFL) sales in company-managed shops rose by **5.5%**, despite a challenging food-to-go market.
   - Improved digital engagement, exemplified by the Greggs App, which accounted for *20.1%* of transactions, enhancing customer loyalty.

3. **Investment in Supply Chain:**
   - Significant investments in infrastructure and technology reflect an ongoing commitment to support expansion and operational efficiencies.
   - The launch of new production lines and capacity handling for increased demand positions the company favorably for future growth.

4. **Evening Trade & Menu Innovation:**
   - The evening segment grew to **9%** of sales, showcasing the company's adaptability through innovative product offerings in newer meal times.

5. **Sustainability Initiatives:**
   - Commitment to reducing environmental impact through various initiatives aligns with modern consumer expectations and enhances brand reputation.

## Risks
1. **Economic Conditions:**
   - Rising inflation and potential changes in consumer spending due to economic uncertainty could impact future sales performance.

2. **Supply Chain Vulnerabilities:**
   - Continued geopolitical tensions and inflationary pressures may disrupt supply chains, impacting raw material costs and product availability.

3. **Competition:**
   - The fast-food sector is highly competitive, and Greggs must continue to innovate to maintain market share against rivals.

4. **Operational Risks:**
   - Business disruption risks (e.g., power outages, site accidents) necessitate robust contingency planning to ensure service continuity.

## Recommendation
### **Buy/Sell Decision: Buy**
- **Rationale:** The recent performance metrics indicate strong operational efficiency, a proactive strategy for growth, and a solid financial foundation. Despite existing market challenges, Greggs is well-positioned to manage risks while pursuing expansion plans. The commitment to sustainability and digital engagement further aligns with contemporary consumer trends, suggesting additional room for growth.

Investors should consider the upward momentum in sales, profit growth, and increasing dividends as indicators of a healthy investment opportunity in Greggs PLC. Transitioning into 2025, the anticipated expansion efforts coupled with robust financial management position the company favorably for sustained performance.

--- 

This report provides a simplified yet comprehensive insight into Greggs PLC for novice investors, establishing a foundation for future investment discussions.